# 機器學習模型建立 - 三高預測

**目標**: 使用處理好的資料建立預測模型

**策略**:
- 使用 T1 + T2 的資料訓練模型
- 預測 T3 時的三高狀態
- 從簡單的 Logistic Regression 開始

**日期**: 2025-11-18

## 1. 載入套件與資料

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# 機器學習套件
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')

# 設定中文字型
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Microsoft YaHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("套件載入成功！")

In [ ]:
# 載入處理好的資料
data_path = Path('../../data/processed/SUA_CVDs_wide_format.csv')
df = pd.read_csv(data_path)

print(f"✅ 資料載入成功！")
print(f"資料形狀: {df.shape[0]:,} 人, {df.shape[1]} 個欄位")
print(f"\n欄位預覽:")
print(df.columns[:10].tolist())

## 2. 準備特徵和目標變數

我們將使用 **T1 和 T2 的資料** 來預測 **T3 的三高狀態**

In [ ]:
# 定義特徵欄位（使用 T1 和 T2 的資料）
# 基礎特徵
t1_features = ['FBG_T1', 'TC_T1', 'Cr_T1', 'UA_T1', 'GFR_T1', 'BMI_T1', 'SBP_T1', 'DBP_T1']
t2_features = ['FBG_T2', 'TC_T2', 'Cr_T2', 'UA_T2', 'GFR_T2', 'BMI_T2', 'SBP_T2', 'DBP_T2']

# Δ 變化特徵（T2 - T1）
delta1_features = ['Delta1_FBG', 'Delta1_TC', 'Delta1_Cr', 'Delta1_UA', 
                   'Delta1_GFR', 'Delta1_BMI', 'Delta1_SBP', 'Delta1_DBP']

# 人口統計特徵
demographic_features = ['sex', 'Age']

# 組合所有特徵
all_features = demographic_features + t1_features + t2_features + delta1_features

print(f"特徵總數: {len(all_features)} 個")
print(f"  - 人口統計: {len(demographic_features)} 個")
print(f"  - T1 特徵: {len(t1_features)} 個")
print(f"  - T2 特徵: {len(t2_features)} 個")
print(f"  - Δ1 特徵: {len(delta1_features)} 個")

In [ ]:
# 定義目標變數（T3 時的三高狀態）
target_columns = ['hypertension_T3', 'hyperglycemia_T3', 'dyslipidemia_T3']

# 準備 X（特徵）和 y（目標）
X = df[all_features]
y_hypertension = df['hypertension_T3']
y_hyperglycemia = df['hyperglycemia_T3']
y_dyslipidemia = df['dyslipidemia_T3']

print("目標變數分佈:")
print(f"\n高血壓 (hypertension_T3):")
print(y_hypertension.value_counts())
print(f"  患病率: {(y_hypertension == 2).mean():.2%}")

print(f"\n高血糖 (hyperglycemia_T3):")
print(y_hyperglycemia.value_counts())
print(f"  患病率: {(y_hyperglycemia == 2).mean():.2%}")

print(f"\n高血脂 (dyslipidemia_T3):")
print(y_dyslipidemia.value_counts())
print(f"  患病率: {(y_dyslipidemia == 2).mean():.2%}")

## 3. 資料分割與標準化

In [ ]:
# 將目標變數轉換為二元分類（1=正常, 2=患病 → 0=正常, 1=患病）
y_hypertension_binary = (y_hypertension == 2).astype(int)
y_hyperglycemia_binary = (y_hyperglycemia == 2).astype(int)
y_dyslipidemia_binary = (y_dyslipidemia == 2).astype(int)

# 分割訓練集和測試集（80% 訓練, 20% 測試）
X_train, X_test, y_train_hp, y_test_hp = train_test_split(
    X, y_hypertension_binary, test_size=0.2, random_state=42, stratify=y_hypertension_binary
)

# 使用相同的分割索引來分割其他目標變數
_, _, y_train_hg, y_test_hg = train_test_split(
    X, y_hyperglycemia_binary, test_size=0.2, random_state=42, stratify=y_hyperglycemia_binary
)
_, _, y_train_dl, y_test_dl = train_test_split(
    X, y_dyslipidemia_binary, test_size=0.2, random_state=42, stratify=y_dyslipidemia_binary
)

print("資料分割完成:")
print(f"訓練集: {X_train.shape[0]} 人")
print(f"測試集: {X_test.shape[0]} 人")
print(f"特徵數: {X_train.shape[1]} 個")

In [ ]:
# 標準化特徵
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ 特徵標準化完成")
print(f"訓練集形狀: {X_train_scaled.shape}")
print(f"測試集形狀: {X_test_scaled.shape}")

## 4. 建立 Baseline 模型 - Logistic Regression

In [ ]:
def train_and_evaluate_model(X_train, X_test, y_train, y_test, model_name, disease_name):
    """訓練模型並評估效能"""
    
    # 建立模型
    if model_name == 'Logistic Regression':
        model = LogisticRegression(random_state=42, max_iter=1000)
    elif model_name == 'Random Forest':
        model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # 訓練模型
    model.fit(X_train, y_train)
    
    # 預測
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # 計算指標
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)
    
    # 顯示結果
    print(f"\n{'='*60}")
    print(f"{disease_name} - {model_name}")
    print(f"{'='*60}")
    print(f"準確率 (Accuracy):  {accuracy:.3f}")
    print(f"精確率 (Precision): {precision:.3f}")
    print(f"召回率 (Recall):    {recall:.3f}")
    print(f"F1 分數:            {f1:.3f}")
    print(f"AUC:                {auc:.3f}")
    
    # 混淆矩陣
    cm = confusion_matrix(y_test, y_pred)
    print(f"\n混淆矩陣:")
    print(f"預測 →    正常  患病")
    print(f"實際 ↓")
    print(f"正常      {cm[0,0]:4d}  {cm[0,1]:4d}")
    print(f"患病      {cm[1,0]:4d}  {cm[1,1]:4d}")
    
    return model, {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc
    }

In [ ]:
# 訓練高血壓預測模型
print("\n" + "#"*60)
print("#" + " "*20 + "高血壓預測模型" + " "*24 + "#")
print("#"*60)

model_hp, metrics_hp = train_and_evaluate_model(
    X_train_scaled, X_test_scaled, 
    y_train_hp, y_test_hp,
    'Logistic Regression',
    '高血壓 (Hypertension)'
)

In [ ]:
# 訓練高血糖預測模型
print("\n" + "#"*60)
print("#" + " "*20 + "高血糖預測模型" + " "*24 + "#")
print("#"*60)

model_hg, metrics_hg = train_and_evaluate_model(
    X_train_scaled, X_test_scaled,
    y_train_hg, y_test_hg,
    'Logistic Regression',
    '高血糖 (Hyperglycemia)'
)

In [ ]:
# 訓練高血脂預測模型
print("\n" + "#"*60)
print("#" + " "*20 + "高血脂預測模型" + " "*24 + "#")
print("#"*60)

model_dl, metrics_dl = train_and_evaluate_model(
    X_train_scaled, X_test_scaled,
    y_train_dl, y_test_dl,
    'Logistic Regression',
    '高血脂 (Dyslipidemia)'
)

## 5. 處理資料不平衡

在 Section 4 中，我們發現了一個嚴重的問題：
- **高血糖模型**: F1 = 0, Recall = 0（完全無法檢測出陽性案例）
- **高血脂模型**: F1 = 0, Recall = 0（完全無法檢測出陽性案例）

這是因為資料存在**嚴重的類別不平衡**問題。讓我們來分析並解決這個問題。

### 5.1 問題發現：嚴重的類別不平衡

讓我們先檢查三個疾病的類別分佈：

In [ ]:
# 檢查訓練集中的類別分佈
print("="*70)
print("訓練集類別分佈分析")
print("="*70)

diseases = {
    '高血壓': y_train_hp,
    '高血糖': y_train_hg,
    '高血脂': y_train_dl
}

for disease_name, y_train in diseases.items():
    n_negative = (y_train == 0).sum()
    n_positive = (y_train == 1).sum()
    total = len(y_train)
    imbalance_ratio = n_negative / n_positive if n_positive > 0 else float('inf')
    
    print(f"\n{disease_name}:")
    print(f"  正常 (0): {n_negative:,} 人 ({n_negative/total*100:.1f}%)")
    print(f"  患病 (1): {n_positive:,} 人 ({n_positive/total*100:.1f}%)")
    print(f"  不平衡比率: {imbalance_ratio:.1f} : 1")
    
    if imbalance_ratio > 10:
        print(f"  ⚠️ 嚴重不平衡！")
    elif imbalance_ratio > 5:
        print(f"  ⚠️ 中度不平衡")
    else:
        print(f"  ✓ 相對平衡")

In [ ]:
# 視覺化類別分佈
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, (disease_name, y_train) in enumerate(diseases.items()):
    counts = pd.Series(y_train).value_counts().sort_index()
    colors = ['#2ecc71', '#e74c3c']  # 綠色=正常, 紅色=患病
    
    axes[idx].bar([0, 1], counts.values, color=colors, alpha=0.7, edgecolor='black')
    axes[idx].set_xlabel('類別 (0=正常, 1=患病)', fontsize=11)
    axes[idx].set_ylabel('樣本數', fontsize=11)
    axes[idx].set_title(f'{disease_name}類別分佈', fontsize=13, fontweight='bold')
    axes[idx].set_xticks([0, 1])
    axes[idx].set_xticklabels(['正常', '患病'])
    axes[idx].grid(axis='y', alpha=0.3)
    
    # 加上數值標籤
    for i, v in enumerate(counts.values):
        axes[idx].text(i, v + max(counts.values)*0.02, f'{v:,}', 
                      ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../../docs/experiments/class_imbalance_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 圖表已儲存")

### 5.2 解決方案 1：使用 class_weight='balanced'

`class_weight='balanced'` 會自動調整類別權重，給予少數類別更高的權重。

**權重計算公式**: $w_i = \frac{n_{samples}}{n_{classes} \times n_{samples\_in\_class\_i}}$

#### 5.2.1 Logistic Regression with class_weight='balanced'

In [ ]:
# 使用 class_weight='balanced' 訓練 Logistic Regression
print("="*70)
print("Logistic Regression with class_weight='balanced'")
print("="*70)

# 儲存結果
lr_balanced_results = {}

# 訓練三個疾病模型
disease_data = {
    '高血壓': (y_train_hp, y_test_hp),
    '高血糖': (y_train_hg, y_test_hg),
    '高血脂': (y_train_dl, y_test_dl)
}

for disease_name, (y_train, y_test) in disease_data.items():
    # 訓練模型
    model = LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced')
    model.fit(X_train_scaled, y_train)
    
    # 預測
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    
    # 儲存結果
    lr_balanced_results[disease_name] = {
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }
    
    # 顯示結果
    print(f"\n{disease_name}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")

#### 5.2.2 Random Forest with class_weight='balanced'

In [ ]:
# 使用 class_weight='balanced' 訓練 Random Forest
print("="*70)
print("Random Forest with class_weight='balanced'")
print("="*70)

# 儲存結果
rf_balanced_results = {}

for disease_name, (y_train, y_test) in disease_data.items():
    # 訓練模型
    model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    model.fit(X_train_scaled, y_train)
    
    # 預測
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    
    # 儲存結果
    rf_balanced_results[disease_name] = {
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }
    
    # 顯示結果
    print(f"\n{disease_name}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")

### 5.3 解決方案 2：使用 SMOTE 過採樣

SMOTE (Synthetic Minority Over-sampling Technique) 透過合成新樣本來平衡資料集。

In [ ]:
# 安裝 imbalanced-learn 套件
import sys
!{sys.executable} -m pip install imbalanced-learn==0.11.0

In [ ]:
# 匯入 SMOTE
from imblearn.over_sampling import SMOTE

print("="*70)
print("使用 SMOTE 過採樣 + Logistic Regression")
print("="*70)

# 儲存結果
smote_results = {}

for disease_name, (y_train, y_test) in disease_data.items():
    print(f"\n{disease_name}:")
    
    # 應用 SMOTE
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)
    
    print(f"  原始訓練集: {X_train_scaled.shape[0]} 樣本")
    print(f"  SMOTE 後:   {X_train_smote.shape[0]} 樣本")
    print(f"  類別分佈 - 正常: {(y_train_smote==0).sum()}, 患病: {(y_train_smote==1).sum()}")
    
    # 訓練模型
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train_smote, y_train_smote)
    
    # 預測（使用原始測試集）
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    
    # 儲存結果
    smote_results[disease_name] = {
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }
    
    # 顯示結果
    print(f"\n  效能指標:")
    print(f"    AUC:       {auc:.3f}")
    print(f"    F1:        {f1:.3f}")
    print(f"    Precision: {precision:.3f}")
    print(f"    Recall:    {recall:.3f}")

### 5.4 比較不同平衡方法的效果

現在讓我們比較所有方法的效能：

In [ ]:
# 整合所有結果 - 收集 baseline 結果
baseline_results = {
    '高血壓': metrics_hp,
    '高血糖': metrics_hg,
    '高血脂': metrics_dl
}

# 建立比較表格
comparison_data = []

for disease in ['高血壓', '高血糖', '高血脂']:
    # Baseline (無平衡處理)
    comparison_data.append({
        '疾病': disease,
        '方法': 'Baseline (無處理)',
        'AUC': baseline_results[disease]['auc'],
        'F1': baseline_results[disease]['f1'],
        'Recall': baseline_results[disease]['recall']
    })
    
    # LR + class_weight='balanced'
    comparison_data.append({
        '疾病': disease,
        '方法': "LR + class_weight='balanced'",
        'AUC': lr_balanced_results[disease]['AUC'],
        'F1': lr_balanced_results[disease]['F1'],
        'Recall': lr_balanced_results[disease]['Recall']
    })
    
    # RF + class_weight='balanced'
    comparison_data.append({
        '疾病': disease,
        '方法': "RF + class_weight='balanced'",
        'AUC': rf_balanced_results[disease]['AUC'],
        'F1': rf_balanced_results[disease]['F1'],
        'Recall': rf_balanced_results[disease]['Recall']
    })
    
    # LR + SMOTE
    comparison_data.append({
        '疾病': disease,
        '方法': 'LR + SMOTE',
        'AUC': smote_results[disease]['AUC'],
        'F1': smote_results[disease]['F1'],
        'Recall': smote_results[disease]['Recall']
    })

comparison_df = pd.DataFrame(comparison_data)

# 顯示完整比較表
print("="*80)
print("不同平衡方法效能比較")
print("="*80)
print(comparison_df.to_string(index=False))

# 為每個疾病找出最佳方法
print("\n" + "="*80)
print("最佳方法總結")
print("="*80)
for disease in ['高血壓', '高血糖', '高血脂']:
    disease_data = comparison_df[comparison_df['疾病'] == disease]
    best_auc = disease_data.loc[disease_data['AUC'].idxmax()]
    best_f1 = disease_data.loc[disease_data['F1'].idxmax()]
    best_recall = disease_data.loc[disease_data['Recall'].idxmax()]
    
    print(f"\n{disease}:")
    print(f"  最佳 AUC:    {best_auc['方法']:30s} (AUC={best_auc['AUC']:.3f})")
    print(f"  最佳 F1:     {best_f1['方法']:30s} (F1={best_f1['F1']:.3f})")
    print(f"  最佳 Recall: {best_recall['方法']:30s} (Recall={best_recall['Recall']:.3f})")

In [ ]:
# 視覺化比較結果
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics_to_plot = ['AUC', 'F1', 'Recall']
colors = ['#3498db', '#e74c3c', '#2ecc71', '#f39c12']

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx]
    
    # 為每個疾病繪製柱狀圖
    diseases = ['高血壓', '高血糖', '高血脂']
    methods = comparison_df['方法'].unique()
    
    x = np.arange(len(diseases))
    width = 0.2
    
    for i, method in enumerate(methods):
        values = []
        for disease in diseases:
            value = comparison_df[(comparison_df['疾病'] == disease) & 
                                 (comparison_df['方法'] == method)][metric].values[0]
            values.append(value)
        
        ax.bar(x + i*width, values, width, label=method, color=colors[i], alpha=0.8)
    
    ax.set_xlabel('疾病類型', fontsize=11)
    ax.set_ylabel(metric, fontsize=11)
    ax.set_title(f'{metric} 比較', fontsize=13, fontweight='bold')
    ax.set_xticks(x + width * 1.5)
    ax.set_xticklabels(diseases)
    ax.legend(fontsize=8, loc='lower right')
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig('../../docs/experiments/imbalance_methods_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 比較圖表已儲存至: docs/experiments/imbalance_methods_comparison.png")

## 6. 準備 Multi-Task Learning (MTL) 資料

**Multi-Task Learning (多工學習)** 是一種機器學習策略，可以同時預測多個相關的目標變數。

在我們的案例中，三高疾病（高血壓、高血糖、高血脂）之間存在生理上的關聯性，因此適合使用 MTL 方法。

**MTL 的優點**:
- 共享特徵表示，提高模型泛化能力
- 利用任務間的相關性，提升預測效能
- 減少訓練時間（一次訓練即可預測三種疾病）

讓我們準備多工學習所需的資料格式。

In [ ]:
# 建立多工學習的目標變數
# 將三個疾病的目標變數組合成一個矩陣
import numpy as np

y_train_multi = np.column_stack([y_train_hp, y_train_hg, y_train_dl])
y_test_multi = np.column_stack([y_test_hp, y_test_hg, y_test_dl])

print("="*70)
print("Multi-Task Learning 資料準備完成")
print("="*70)
print(f"\n訓練集 Multi-task 目標形狀: {y_train_multi.shape}")
print(f"測試集 Multi-task 目標形狀: {y_test_multi.shape}")
print(f"\n每個樣本包含 3 個目標變數: [高血壓, 高血糖, 高血脂]")
print(f"\n範例（前 5 個訓練樣本）:")
print(f"{'樣本':>6s}  {'高血壓':>6s}  {'高血糖':>6s}  {'高血脂':>6s}")
print("-" * 35)
for i in range(5):
    print(f"{i+1:6d}  {y_train_multi[i, 0]:6d}  {y_train_multi[i, 1]:6d}  {y_train_multi[i, 2]:6d}")
    
print("\n✅ MTL 資料已準備完成，可以開始訓練多工學習模型！")

## 7. MTL 方法 1 - MultiOutputClassifier

`MultiOutputClassifier` 是 scikit-learn 提供的多輸出分類器，可以將任何單輸出分類器轉換為多輸出版本。

**工作原理**: 為每個目標變數訓練一個獨立的分類器，但共享相同的特徵空間。

讓我們分別使用 Logistic Regression 和 Random Forest 作為基礎分類器。

### 7.1 MTL Logistic Regression (without balancing)

In [ ]:
from sklearn.multioutput import MultiOutputClassifier

print("="*70)
print("MTL 方法 1: MultiOutputClassifier + Logistic Regression")
print("="*70)

# 建立多輸出 Logistic Regression 模型
mtl_lr = MultiOutputClassifier(
    LogisticRegression(random_state=42, max_iter=1000)
)

# 訓練模型
mtl_lr.fit(X_train_scaled, y_train_multi)

# 預測
y_pred_mtl_lr = mtl_lr.predict(X_test_scaled)
y_pred_proba_mtl_lr = mtl_lr.predict_proba(X_test_scaled)

# 疾病名稱
disease_names = ['高血壓', '高血糖', '高血脂']

# 計算每個疾病的效能指標
print("\n效能評估:")
print("-" * 70)

mtl_lr_results = {}
for i, disease_name in enumerate(disease_names):
    y_true = y_test_multi[:, i]
    y_pred = y_pred_mtl_lr[:, i]
    y_proba = y_pred_proba_mtl_lr[i][:, 1]  # 第 i 個分類器的正類別機率
    
    # 計算指標
    auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    mtl_lr_results[disease_name] = {
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }
    
    # 混淆矩陣
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{disease_name}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

print("\n✅ MTL Logistic Regression 訓練完成！")

### 7.2 MTL Random Forest (native multi-output)

**注意**: Random Forest 天生支援多輸出，不需要使用 `MultiOutputClassifier` 包裝器。

In [ ]:
print("="*70)
print("MTL 方法 1: Random Forest (native multi-output)")
print("="*70)

# Random Forest 原生支援多輸出
mtl_rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# 訓練模型
mtl_rf.fit(X_train_scaled, y_train_multi)

# 預測
y_pred_mtl_rf = mtl_rf.predict(X_test_scaled)
y_pred_proba_mtl_rf = mtl_rf.predict_proba(X_test_scaled)

# 計算每個疾病的效能指標
print("\n效能評估:")
print("-" * 70)

mtl_rf_results = {}
for i, disease_name in enumerate(disease_names):
    y_true = y_test_multi[:, i]
    y_pred = y_pred_mtl_rf[:, i]
    # 注意：RF 的 predict_proba 返回的是列表，每個元素是一個疾病的機率矩陣
    y_proba = y_pred_proba_mtl_rf[i][:, 1]  # 第 i 個疾病的正類別機率
    
    # 計算指標
    auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    mtl_rf_results[disease_name] = {
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }
    
    # 混淆矩陣
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{disease_name}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

print("\n✅ MTL Random Forest 訓練完成！")

## 8. MTL 方法 2 - ClassifierChain

`ClassifierChain` 是一種更進階的多輸出學習方法，它會將前一個任務的預測結果作為下一個任務的特徵。

**工作原理**:
1. 按照指定順序訓練分類器
2. 每個分類器都會使用前面分類器的預測結果作為額外特徵
3. 可以捕捉任務之間的依賴關係

**疾病順序**: 我們使用 [0, 2, 1]，即 **高血壓 → 高血脂 → 高血糖**

### 8.1 ClassifierChain with Logistic Regression

In [ ]:
from sklearn.multioutput import ClassifierChain

print("="*70)
print("MTL 方法 2: ClassifierChain + Logistic Regression")
print("="*70)

# 建立 ClassifierChain 模型
# order=[0, 2, 1] 表示預測順序: 高血壓 → 高血脂 → 高血糖
chain_lr = ClassifierChain(
    LogisticRegression(random_state=42, max_iter=1000),
    order=[0, 2, 1],
    random_state=42
)

# 訓練模型
chain_lr.fit(X_train_scaled, y_train_multi)

# 預測
y_pred_chain_lr = chain_lr.predict(X_test_scaled)
# ClassifierChain 不直接支援 predict_proba，需要手動獲取
# 我們使用 decision_function 或個別分類器的 predict_proba

# 計算每個疾病的效能指標
print("\n效能評估:")
print("-" * 70)

chain_lr_results = {}
for i, disease_name in enumerate(disease_names):
    y_true = y_test_multi[:, i]
    y_pred = y_pred_chain_lr[:, i]
    
    # 獲取該疾病分類器的機率預測
    # 找到在 chain 中的位置
    chain_idx = list(chain_lr.order_).index(i)
    y_proba = chain_lr.estimators_[chain_idx].predict_proba(
        X_test_scaled if chain_idx == 0 else 
        np.column_stack([X_test_scaled, y_pred_chain_lr[:, :chain_idx]])
    )[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    chain_lr_results[disease_name] = {
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }
    
    # 混淆矩陣
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{disease_name}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

print("\n✅ ClassifierChain + Logistic Regression 訓練完成！")

### 8.2 ClassifierChain with Random Forest

In [ ]:
print("="*70)
print("MTL 方法 2: ClassifierChain + Random Forest")
print("="*70)

# 建立 ClassifierChain 模型
chain_rf = ClassifierChain(
    RandomForestClassifier(n_estimators=100, random_state=42),
    order=[0, 2, 1],
    random_state=42
)

# 訓練模型
chain_rf.fit(X_train_scaled, y_train_multi)

# 預測
y_pred_chain_rf = chain_rf.predict(X_test_scaled)

# 計算每個疾病的效能指標
print("\n效能評估:")
print("-" * 70)

chain_rf_results = {}
for i, disease_name in enumerate(disease_names):
    y_true = y_test_multi[:, i]
    y_pred = y_pred_chain_rf[:, i]
    
    # 獲取該疾病分類器的機率預測
    chain_idx = list(chain_rf.order_).index(i)
    y_proba = chain_rf.estimators_[chain_idx].predict_proba(
        X_test_scaled if chain_idx == 0 else 
        np.column_stack([X_test_scaled, y_pred_chain_rf[:, :chain_idx]])
    )[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    chain_rf_results[disease_name] = {
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }
    
    # 混淆矩陣
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{disease_name}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

print("\n✅ ClassifierChain + Random Forest 訓練完成！")

## 9. MTL with class_weight='balanced'

前面的 MTL 方法沒有處理類別不平衡問題。現在讓我們將 `class_weight='balanced'` 應用到 MTL 模型中，以改善對少數類別（患病案例）的檢測能力。

### 9.1 MTL Logistic Regression with balanced weights

In [ ]:
print("="*70)
print("MTL Logistic Regression with class_weight='balanced'")
print("="*70)

# 建立多輸出 Logistic Regression 模型（使用 balanced 權重）
mtl_lr_balanced = MultiOutputClassifier(
    LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
)

# 訓練模型
mtl_lr_balanced.fit(X_train_scaled, y_train_multi)

# 預測
y_pred_mtl_lr_bal = mtl_lr_balanced.predict(X_test_scaled)
y_pred_proba_mtl_lr_bal = mtl_lr_balanced.predict_proba(X_test_scaled)

# 計算每個疾病的效能指標
print("\n效能評估:")
print("-" * 70)

mtl_lr_balanced_results = {}
for i, disease_name in enumerate(disease_names):
    y_true = y_test_multi[:, i]
    y_pred = y_pred_mtl_lr_bal[:, i]
    y_proba = y_pred_proba_mtl_lr_bal[i][:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    mtl_lr_balanced_results[disease_name] = {
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }
    
    # 混淆矩陣
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{disease_name}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

print("\n✅ MTL LR (balanced) 訓練完成！")

### 9.2 MTL Random Forest with balanced weights

In [ ]:
print("="*70)
print("MTL Random Forest with class_weight='balanced'")
print("="*70)

# Random Forest 原生支援多輸出，並使用 balanced 權重
mtl_rf_balanced = RandomForestClassifier(
    n_estimators=100, 
    class_weight='balanced',
    random_state=42, 
    n_jobs=-1
)

# 訓練模型
mtl_rf_balanced.fit(X_train_scaled, y_train_multi)

# 預測
y_pred_mtl_rf_bal = mtl_rf_balanced.predict(X_test_scaled)
y_pred_proba_mtl_rf_bal = mtl_rf_balanced.predict_proba(X_test_scaled)

# 計算每個疾病的效能指標
print("\n效能評估:")
print("-" * 70)

mtl_rf_balanced_results = {}
for i, disease_name in enumerate(disease_names):
    y_true = y_test_multi[:, i]
    y_pred = y_pred_mtl_rf_bal[:, i]
    y_proba = y_pred_proba_mtl_rf_bal[i][:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    mtl_rf_balanced_results[disease_name] = {
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }
    
    # 混淆矩陣
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{disease_name}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

print("\n✅ MTL RF (balanced) 訓練完成！")

### 9.3 ClassifierChain with balanced weights

In [ ]:
print("="*70)
print("ClassifierChain with class_weight='balanced'")
print("="*70)

# 建立 ClassifierChain 模型（使用 balanced 權重的 RF）
chain_rf_balanced = ClassifierChain(
    RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    order=[0, 2, 1],
    random_state=42
)

# 訓練模型
chain_rf_balanced.fit(X_train_scaled, y_train_multi)

# 預測
y_pred_chain_rf_bal = chain_rf_balanced.predict(X_test_scaled)

# 計算每個疾病的效能指標
print("\n效能評估:")
print("-" * 70)

chain_rf_balanced_results = {}
for i, disease_name in enumerate(disease_names):
    y_true = y_test_multi[:, i]
    y_pred = y_pred_chain_rf_bal[:, i]
    
    # 獲取該疾病分類器的機率預測
    chain_idx = list(chain_rf_balanced.order_).index(i)
    y_proba = chain_rf_balanced.estimators_[chain_idx].predict_proba(
        X_test_scaled if chain_idx == 0 else 
        np.column_stack([X_test_scaled, y_pred_chain_rf_bal[:, :chain_idx]])
    )[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_true, y_proba)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    precision = precision_score(y_true, y_pred, zero_division=0)
    recall = recall_score(y_true, y_pred, zero_division=0)
    
    chain_rf_balanced_results[disease_name] = {
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }
    
    # 混淆矩陣
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{disease_name}:")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

print("\n✅ ClassifierChain (balanced) 訓練完成！")

### 9.4 MTL 方法比較與視覺化

現在讓我們比較所有 MTL 方法的效能，包括有無 class_weight='balanced' 的版本。

In [ ]:
# 整合所有 MTL 方法的結果
mtl_comparison_data = []

# 所有 MTL 方法
mtl_methods = {
    'MTL LR': mtl_lr_results,
    'MTL RF': mtl_rf_results,
    'Chain LR': chain_lr_results,
    'Chain RF': chain_rf_results,
    'MTL LR (balanced)': mtl_lr_balanced_results,
    'MTL RF (balanced)': mtl_rf_balanced_results,
    'Chain RF (balanced)': chain_rf_balanced_results
}

# 建立比較表格
for method_name, results in mtl_methods.items():
    for disease_name in disease_names:
        mtl_comparison_data.append({
            '方法': method_name,
            '疾病': disease_name,
            'AUC': results[disease_name]['AUC'],
            'F1': results[disease_name]['F1'],
            'Recall': results[disease_name]['Recall']
        })

mtl_comparison_df = pd.DataFrame(mtl_comparison_data)

# 顯示完整比較表
print("="*80)
print("MTL 方法效能比較")
print("="*80)
print(mtl_comparison_df.to_string(index=False))

# 為每個疾病找出最佳方法
print("\n" + "="*80)
print("最佳 MTL 方法總結")
print("="*80)
for disease in disease_names:
    disease_data = mtl_comparison_df[mtl_comparison_df['疾病'] == disease]
    best_auc = disease_data.loc[disease_data['AUC'].idxmax()]
    best_f1 = disease_data.loc[disease_data['F1'].idxmax()]
    best_recall = disease_data.loc[disease_data['Recall'].idxmax()]
    
    print(f"\n{disease}:")
    print(f"  最佳 AUC:    {best_auc['方法']:25s} (AUC={best_auc['AUC']:.3f})")
    print(f"  最佳 F1:     {best_f1['方法']:25s} (F1={best_f1['F1']:.3f})")
    print(f"  最佳 Recall: {best_recall['方法']:25s} (Recall={best_recall['Recall']:.3f})")

In [ ]:
# 視覺化 MTL 方法比較
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

metrics_to_plot = ['AUC', 'F1', 'Recall']
colors = plt.cm.tab10(range(len(mtl_methods)))

for idx, metric in enumerate(metrics_to_plot):
    ax = axes[idx]
    
    # 為每個疾病繪製柱狀圖
    x = np.arange(len(disease_names))
    width = 0.12  # 更窄的柱子以容納更多方法
    
    for i, method in enumerate(mtl_methods.keys()):
        values = []
        for disease in disease_names:
            value = mtl_comparison_df[(mtl_comparison_df['疾病'] == disease) & 
                                     (mtl_comparison_df['方法'] == method)][metric].values[0]
            values.append(value)
        
        ax.bar(x + i*width, values, width, label=method, color=colors[i], alpha=0.8)
    
    ax.set_xlabel('疾病類型', fontsize=11)
    ax.set_ylabel(metric, fontsize=11)
    ax.set_title(f'MTL 方法 {metric} 比較', fontsize=13, fontweight='bold')
    ax.set_xticks(x + width * 3)
    ax.set_xticklabels(disease_names)
    ax.legend(fontsize=7, loc='lower right', ncol=2)
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig('../../docs/experiments/mtl_methods_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ MTL 比較圖表已儲存至: docs/experiments/mtl_methods_comparison.png")

## 10. 比較不同特徵組合的效果

In [ ]:
# 定義不同的特徵組合
feature_sets = {
    '只用 T1 特徵': demographic_features + t1_features,
    'T1 + Δ1 特徵': demographic_features + t1_features + delta1_features,
    '完整特徵 (T1+T2+Δ1)': all_features
}

# 儲存結果
results = {}

print("\n" + "="*80)
print("比較不同特徵組合 - 高血壓預測")
print("="*80)

for name, features in feature_sets.items():
    # 準備資料
    X_subset = df[features]
    X_train_subset, X_test_subset, y_train, y_test = train_test_split(
        X_subset, y_hypertension_binary, test_size=0.2, random_state=42, stratify=y_hypertension_binary
    )
    
    # 標準化
    scaler = StandardScaler()
    X_train_subset = scaler.fit_transform(X_train_subset)
    X_test_subset = scaler.transform(X_test_subset)
    
    # 訓練模型
    model = LogisticRegression(random_state=42, max_iter=1000)
    model.fit(X_train_subset, y_train)
    
    # 預測和評估
    y_pred = model.predict(X_test_subset)
    y_pred_proba = model.predict_proba(X_test_subset)[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred)
    
    results[name] = {'AUC': auc, 'F1': f1}
    
    print(f"\n{name} ({len(features)} 個特徵):")
    print(f"  AUC: {auc:.3f}")
    print(f"  F1:  {f1:.3f}")

## 11. 視覺化模型效能比較

In [ ]:
# 整合三個疾病的預測結果
all_metrics = pd.DataFrame({
    '高血壓': metrics_hp,
    '高血糖': metrics_hg,
    '高血脂': metrics_dl
}).T

# 繪製比較圖
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 圖1: 各項指標比較
all_metrics[['accuracy', 'precision', 'recall', 'f1']].plot(kind='bar', ax=axes[0])
axes[0].set_title('三高預測模型效能比較', fontsize=14, fontweight='bold')
axes[0].set_xlabel('疾病類型')
axes[0].set_ylabel('分數')
axes[0].set_ylim([0, 1])
axes[0].legend(['準確率', '精確率', '召回率', 'F1分數'])
axes[0].grid(axis='y', alpha=0.3)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# 圖2: AUC 比較
all_metrics['auc'].plot(kind='bar', ax=axes[1], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
axes[1].set_title('AUC 分數比較', fontsize=14, fontweight='bold')
axes[1].set_xlabel('疾病類型')
axes[1].set_ylabel('AUC')
axes[1].set_ylim([0.5, 1])
axes[1].grid(axis='y', alpha=0.3)
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

# 在柱狀圖上加上數值標籤
for i, v in enumerate(all_metrics['auc']):
    axes[1].text(i, v + 0.01, f'{v:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../../docs/experiments/model_performance_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 圖表已儲存至: docs/experiments/model_performance_comparison.png")

## 12. 嘗試 Random Forest 模型

In [ ]:
print("\n" + "#"*60)
print("#" + " "*15 + "Random Forest 模型測試" + " "*19 + "#")
print("#"*60)

# 使用 Random Forest 預測高血壓
rf_model_hp, rf_metrics_hp = train_and_evaluate_model(
    X_train_scaled, X_test_scaled,
    y_train_hp, y_test_hp,
    'Random Forest',
    '高血壓 (Hypertension)'
)

# 比較 Logistic Regression vs Random Forest
print("\n" + "="*60)
print("模型比較總結 - 高血壓預測")
print("="*60)
print(f"\nLogistic Regression:")
print(f"  AUC: {metrics_hp['auc']:.3f}")
print(f"  F1:  {metrics_hp['f1']:.3f}")
print(f"\nRandom Forest:")
print(f"  AUC: {rf_metrics_hp['auc']:.3f}")
print(f"  F1:  {rf_metrics_hp['f1']:.3f}")
print(f"\n改善幅度:")
print(f"  AUC: {(rf_metrics_hp['auc'] - metrics_hp['auc'])*100:+.1f}%")
print(f"  F1:  {(rf_metrics_hp['f1'] - metrics_hp['f1'])*100:+.1f}%")

## 13. 特徵重要性分析

In [ ]:
# 取得 Random Forest 的特徵重要性
feature_importance = pd.DataFrame({
    'feature': all_features,
    'importance': rf_model_hp.feature_importances_
}).sort_values('importance', ascending=False)

# 顯示前 15 個最重要的特徵
print("="*60)
print("特徵重要性排名 (Top 15)")
print("="*60)
for i, row in feature_importance.head(15).iterrows():
    print(f"{row['feature']:20s} {row['importance']:.4f}")

# 視覺化
plt.figure(figsize=(10, 6))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['importance'], color='skyblue')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('重要性分數')
plt.title('特徵重要性排名 - 高血壓預測', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../../docs/experiments/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ 圖表已儲存至: docs/experiments/feature_importance.png")

## 14. 總結

### 🎯 完成的工作
1. ✅ 成功建立了三高（高血壓、高血糖、高血脂）的預測模型
2. ✅ 使用 T1 + T2 的資料預測 T3 時的疾病狀態
3. ✅ 比較了 Logistic Regression 和 Random Forest 兩種模型
4. ✅ 分析了特徵重要性
5. ✅ 識別並解決了嚴重的類別不平衡問題
6. ✅ 比較了多種平衡方法（class_weight='balanced' 和 SMOTE）

### 📊 模型效能
- **Logistic Regression**: 提供了穩定的 baseline 結果
- **Random Forest**: 通常能獲得更好的預測效能
- **最重要的特徵**: 通常是 T2 時間點的測量值和 Δ 變化量
- **類別不平衡處理**: class_weight='balanced' 和 SMOTE 都能顯著改善少數類別的檢測能力

### 💡 下一步建議
1. **嘗試 XGBoost**: 通常能獲得最佳效能
2. **超參數調優**: 使用 GridSearchCV 或 RandomizedSearchCV
3. **特徵工程**: 創建交互特徵或多項式特徵
4. **集成學習**: 結合多個模型的預測結果
5. **時序分析**: 考慮使用 LSTM 等時序模型